In [2]:
!git clone https://github.com/Abhiram0908/Helmet-Number-Plate-Detection.git

Cloning into 'Helmet-Number-Plate-Detection'...
remote: Enumerating objects: 167, done.
remote: Counting objects: 100% (47/47), done.
remote: Compressing objects: 100% (34/34), done.
remote: Total 167 (delta 18), reused 41 (delta 12), pack-reused 120
Receiving objects: 100% (167/167), 192.49 MiB | 31.67 MiB/s, done.
Resolving deltas: 100% (38/38), done.


In [3]:
cd /content/Helmet-Number-Plate-Detection

/content/Helmet-Number-Plate-Detection


In [1]:
pip install -r requirements.txt

  Using cached ultralytics-8.0.3-py3-none-any.whl (247 kB)
  Using cached hydra_core-1.3.2-py3-none-any.whl (154 kB)
  Using cached omegaconf-2.3.0-py3-none-any.whl (79 kB)
  Using cached numpy-1.25.2-cp310-cp310-win_amd64.whl (15.6 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
  Rolling back uninstall of numpy
  Moving to c:\users\abhiram dacharla\appdata\roaming\python\python310\scripts\f2py.exe
   from C:\Users\Abhiram Dacharla\AppData\Local\Temp\pip-uninstall-3i8bhug5\f2py.exe
  Moving to c:\users\abhiram dacharla\appdata\roaming\python\python310\site-packages\numpy-1.26.4-cp310-cp310-win_amd64.whl
   from C:\Users\Abhiram Dacharla\AppData\Local\Temp\pip-uninstall-m6oqg_sz\numpy-1.26.4-cp310-cp310-win_amd64.whl
  Moving to c:\users\abhiram dacharla\appdata\roaming\python\python310\site-packages\numpy-1.26.4.dist-info\
   from C:\Users\Abhiram Dacharla\AppData\Roaming\Python\

ERROR: Could not install packages due to an OSError: [WinError 2] The system cannot find the file specified: 'c:\\Python310\\Scripts\\f2py.exe' -> 'c:\\Python310\\Scripts\\f2py.exe.deleteme'



In [3]:
! pip install tesseract
! pip install Pillow
! pip install pytesseract

Defaulting to user installation because normal site-packages is not writeable


You should consider upgrading via the 'c:\program files\python38\python.exe -m pip install --upgrade pip' command.


Defaulting to user installation because normal site-packages is not writeable


You should consider upgrading via the 'c:\program files\python38\python.exe -m pip install --upgrade pip' command.


Defaulting to user installation because normal site-packages is not writeable


You should consider upgrading via the 'c:\program files\python38\python.exe -m pip install --upgrade pip' command.


In [4]:
from ultralytics import YOLO
import cv2
import pytesseract
import os
import sqlite3

ModuleNotFoundError: No module named 'hydra'

In [7]:
# Function to create the table if it doesn't exist
def create_table():
    # Connect to the SQLite database
    conn = sqlite3.connect("vehicle_data.db")
    # Create a cursor object to execute SQL statements
    c = conn.cursor()
    # Create the "vehicles" table with two columns: "vehicle_number" and "bike_image_path"
    # The "IF NOT EXISTS" clause ensures that the table is only created if it doesn't already exist
    c.execute("CREATE TABLE IF NOT EXISTS vehicles (vehicle_number TEXT, bike_image_path TEXT)")
    # Commit the changes to the database
    conn.commit()
    # Close the database connection
    conn.close()

# Function to insert a record into the "vehicles" table
def insert_record(vehicle_number, bike_image_path):
    # Connect to the SQLite database
    conn = sqlite3.connect("vehicle_data.db")
    # Create a cursor object to execute SQL statements
    c = conn.cursor()
    # Insert the record into the "vehicles" table using parameterized SQL statement
    c.execute("INSERT INTO vehicles VALUES (?, ?)", (vehicle_number, bike_image_path))
    # Commit the changes to the database
    conn.commit()
    # Close the database connection
    conn.close()

In [8]:
# Initialize YOLO models
person_bike_model = YOLO(r"/content/Helmet-Number-Plate-Detection/Phase-1.pt")
helmet_model = YOLO(r"/content/Helmet-Number-Plate-Detection/Phase-2.pt")
number_plate_model = YOLO(r"/content/Helmet-Number-Plate-Detection/Phase-3.pt")

In [9]:
# Set up Tesseract OCR
pytesseract.pytesseract.tesseract_cmd = r"/content/Helmet-Number-Plate-Detection/tesseract-ocr-setup-3.02.02.exe"  # Update with the path to your Tesseract OCR executable

In [11]:
# Directory to save the output images
output_dir = r"/content/Helmet-Number-Plate-Detection"

In [12]:
# Set up video capture
video_capture = cv2.VideoCapture(r"/content/Helmet-Number-Plate-Detection/helmet_demo.mp4")  # Use 0 for the default camera or provide the desired camera index

while True:
    # Capture frame-by-frame
    ret, frame = video_capture.read()
    # Process frame
    img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    # Detect person on a bike
    person_bike_results = person_bike_model.predict(img)

    # Process each detection result
    print("Bike Result")
    for r in person_bike_results:
        boxes = r.boxes
        # Filter detections for person on a bike
        for box in boxes:
            cls = box.cls
            print(person_bike_model.names[int(cls)], person_bike_model.names[int(cls)] == "Person_Bikes")
            if person_bike_model.names[int(cls)] == "Person_Bike":
                # Crop person on a bike image
                x1, y1, x2, y2 = box.xyxy[0]
                person_bike_image = frame[int(y1):int(y2), int(x1):int(x2)]

                # Detect helmet on the person
                helmet_results = helmet_model.predict(person_bike_image)

                # Process each helmet detection result
                for hr in helmet_results:
                    h_boxes = hr.boxes
                    # Filter detections for no helmet
                    for h_bo in h_boxes:
                        h_cls = h_bo.cls
                        if not helmet_model.names[int(h_cls)] == "With Helmet" :
                            print("Person wihout Element")
                            # Extract number plate from the person bike image
                            number_plate_results = number_plate_model.predict(person_bike_image)

                            # Process each number plate detection result
                            for npr in number_plate_results:
                                np_boxes = npr.boxes
                                # Filter detections for number plate
                                for np_box in np_boxes:
                                    np_cls = np_box.cls
                                    print(number_plate_model.names[int(np_cls)])
                                    if number_plate_model.names[int(np_cls)] == "number_plate":
                                        # Crop number plate image
                                        np_x1, np_y1, np_x2, np_y2 = np_box.xyxy[0]
                                        number_plate_image = person_bike_image[int(np_y1):int(np_y2),
                                                             int(np_x1):int(np_x2)]
                                        # Save the cropped number plate image
                                        output_file = f"person_violation_{image_file}"
                                        output_path = os.path.join(output_dir, output_file)
                                        cv2.imwrite(output_path, person_bike_image)

                                        # Perform OCR on the number plate image
                                        gray = cv2.cvtColor(number_plate_image, cv2.COLOR_BGR2GRAY)
                                        text = pytesseract.image_to_string(gray)
                                        # Example usage
                                        # Create the "vehicles" table if it doesn't exist
                                        create_table()
                                        # Insert two records into the "vehicles" table
                                        insert_record(text, output_path)
                                        # Print the extracted text
                                        print("Number Plate Text:", text)

Ultralytics YOLOv8.0.3 🚀 Python-3.10.12 torch-2.2.1+cu121 CPU
Fusing layers... 
Model summary: 218 layers, 25840339 parameters, 0 gradients, 78.7 GFLOPs


FileNotFoundError: /content/Helmet-Number-Plate-Detection/[[[176 200 220]
  [176 200 220]
  [176 200 220]
  ...
  [174 193 214]
  [174 193 214]
  [174 193 214]]

 [[176 200 220]
  [176 200 220]
  [176 200 220]
  ...
  [174 193 214]
  [174 193 214]
  [174 193 214]]

 [[176 200 220]
  [176 200 220]
  [176 200 220]
  ...
  [174 193 214]
  [174 193 214]
  [174 193 214]]

 ...

 [[127 123 117]
  [127 123 117]
  [127 123 117]
  ...
  [214 192 174]
  [214 192 174]
  [214 192 174]]

 [[127 123 117]
  [127 123 117]
  [127 123 117]
  ...
  [214 192 174]
  [214 192 174]
  [214 192 174]]

 [[127 123 117]
  [127 123 117]
  [127 123 117]
  ...
  [214 192 174]
  [214 192 174]
  [214 192 174]]] does not exist